In [1]:
import cv2
import numpy as np
import os
from sklearn.cluster import DBSCAN

In [2]:
def extract_and_match_features(image1_path, image2_path):
    # 读取图像
    img1 = cv2.imread(image1_path)
    img2 = cv2.imread(image2_path)

    # 创建SIFT特征提取器
    sift = cv2.SIFT_create()

    # 提取特征点和描述子
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # 创建暴力匹配器
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # 筛选出好的匹配点（使用Lowe's ratio test）
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    return good_matches, kp1, kp2

In [3]:
def get_object_locations(good_matches, kp1, kp2):
    locations1 = []
    locations2 = []
    for match in good_matches:
        pt1 = kp1[match.queryIdx].pt
        pt2 = kp2[match.trainIdx].pt
        locations1.append(pt1)
        locations2.append(pt2)

    return np.array(locations1), np.array(locations2)

In [4]:
def cluster_objects(locations_all):
    clustering = DBSCAN(eps=10, min_samples=2).fit(locations_all)
    return clustering.labels_

In [5]:
def count_unique_objects(labels):
    unique_labels = set(labels)
    num_unique_objects = len(unique_labels) - (1 if -1 in unique_labels else 0)  # 排除噪声点（标记为-1的情况）
    return num_unique_objects

In [6]:
def count_objects_in_images(image_folder):
    image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.jpg') or f.endswith('.png')]
    locations_all = []
    for i in range(len(image_files) - 1):
        for j in range(i + 1, len(image_files)):
            good_matches, kp1, kp2 = extract_and_match_features(image_files[i], image_files[j])
            locations1, locations2 = get_object_locations(good_matches, kp1, kp2)
            locations_all.extend(locations1)
            locations_all.extend(locations2)

    labels = cluster_objects(np.array(locations_all))
    num_unique_objects = count_unique_objects(labels)
    return num_unique_objects

In [9]:
num_objects = count_objects_in_images('./mini')
print(f"不重复的物体个数为: {num_objects}")

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 85229568 bytes in function 'cv::OutOfMemoryError'
